# Goal: students will implement a basic relational algebra system using Python.

## Loading Data

We will read from csv files into lists

In [ ]:
import csv
def open_file(filename):
    with open(filename, 'r', newline = '') as f:
        reader = csv.reader(f, delimiter='\t')
        data = list(reader)
        return data[1:]

In [ ]:
# Data source: https://relational.fit.cvut.cz/dataset/IMDb
# Information courtesy of IMDb (http://www.imdb.com). Used with permission.
# Notice: The data can only be used for personal and non-commercial use and must not
# be altered/republished/resold/repurposed to create any kind of online/offline
# database of movie information (except for individual personal use).

path = 'data/IMDb_sample'  
# create a list of all files in that directory that end with "*.csv":

actors = open_file(path + '/actors.csv')
directors_genres = open_file(path + '/directors_genres.csv')
directors = open_file(path + '/directors.csv')
movies_directors = open_file(path + '/movies_directors.csv')
movies_genres = open_file(path + '/movies_genres.csv')
movies = open_file(path + '/movies.csv')
roles = open_file(path + '/roles.csv')


In [ ]:
actors

In [ ]:
# Creating another table with actors for testing purposes

extra_actors = [['109100', 'Renata', 'Dividino', 'F'], ['481290', 'Burnell', 'Tucker', 'M'], ['10963', 'Chris', 'Anastasio', 'M']]
extra_actors

## Implementing Relational Algebra Operators

### Set Operators

In [ ]:
def union(rel_a, rel_b):
    """Relational algebra project set union.
    Finds the union between two relations.
    
    Parameters:
        rel_a : the first relation.
        rel_b : the second relation.
    
    Returns:
        The union between rel_a and rel_b.
    """ 
    return rel_a + rel_b

In [ ]:
def union2(rel_a, rel_b):
    """Relational algebra project set union with no duplicates.
    Finds the union between two relations, but without duplicates.

    Parameters:
        rel_a : the first relation.
        rel_b : the second relation.
    
    Returns:
        The union between rel_a and rel_b without reduplication.
    """
    # effectively cast the union as a set, which has no duplicates.
    return set(union(rel_a, rel_b))

In [ ]:
def intersection(rel_a, rel_b):
    """Relational algebra project set intersection.
    Finds the intersection between two relations.
    
    Parameters:
        rel_a : the first relation.
        rel_b : the second relation.
    
    Returns:
        The intersect between rel_a and rel_b.
    """ 
    return [i for i in rel_a if i in rel_b]

In [ ]:
def set_difference(rel_a, rel_b):
    """Relational algebra project set difference.
    Finds the set difference between two relations.
    
    Parameters:
        rel_a : the first relation.
        rel_b : the second relation.
    
    Returns:
        The set difference between rel_a and rel_b.
    """ 
    return [i for i in rel_a if i not in rel_b]

Now Perform the union between the actors and extra_actors relation
This operation should returns all of the rows of actor and extra_actors unioned in to a single list. 

In [ ]:
union(actors, extra_actors)

### Selection

Now we can use a loop to perform selections on our data. For example we can create a list of all of the male actors:

In [ ]:
import operator # needed to map operators to modes.

def selection(relation, column, predicate, mode):
    """Relational algebra project selection.
    Finds rows where some selection criteria is met.
    
    Parameters:
        relation : the relation to apply selection operator on.
        column : the column to select from.
        predicate : what to select from column.
        mode : what kind of arithmetic operator to use for selection.
    
    Returns:
        A relation of rows as selected from the original relation.    
    """ 
    # a dict mapping operators to modes
    op_map = {
        '<':operator.lt,
        '>':operator.gt,
        '<=':operator.le,
        '>=':operator.ge,
        '==':operator.eq,
        '!=':operator.ne
    }
    return [i for i in relation if op_map[mode](i[column], predicate)]

In [ ]:
# perform selection on actors data, gender is 3rd (zero indexed) column.
selection(actors, 3, 'M', '==')

Exercise:
- Show all tuples from actors that first name is Chris
- Show all tuples from movies that were made after 1998 
- Show all tuples from actors that are female AND id is bigger than 200000

In [ ]:
selection(actors, 1, 'Chris', '==')

In [ ]:
selection(movies, 2, '1998', '>')

In [ ]:
# find relations as a function of the selection operator applied to a table.
rel_a = selection(actors, 3, 'F', '==')
rel_b = selection(actors, 0, '200000', '>')

# intersect the two.
intersection(rel_a, rel_b)

### Projection

Now we want to implement the projection operator.

In [ ]:
def project(relation, columns):
    """Relational algebra projection operator.
    Projects specified columns from the relation to then be viewed.
    
    Parameters:
       relation : the relation to project.
       columns : list iterable columns to project from relation.
    
    Returns:
        The specified columns from the relation.
    """
    # catch instances where supplied column parameter is out of range.
    try:
        proj = [[row[i] for i in columns] for row in relation]
    except IndexError:
        proj = []
    return proj

Exercise:

- Find the last name of all actors named Chris
- Find the names of the movies that were made after 1998 
- Find the name of actors that are female AND id is bigger than 200000

In [ ]:
# apply selection operator to table, then project specified column.
project(selection(actors, 1, 'Chris', '=='), [2])

In [ ]:
# apply selection operator to table, then project specified column.
project(selection(movies, 2, '1998', '>'), [1])

In [ ]:
# find relations as a function of the selection operator applied to a table.
rel_a = selection(actors, 3, 'F' ,'==')
rel_b = selection(actors, 0, '200000' ,'>')

# intersect the two and project the specified column.
project(intersection(rel_a, rel_b), [1])

### Cross Product

Lets implement the cross product next. The cross product pairs each row of a relation with every row of another relation to create a new relation that contains every possible combination of the input relations tuples.

In [ ]:
def crossproduct(rel_a, rel_b):
    """Finds the cross product between two relations.
    The cross product between two relations is the concatenation of each
    row in the first relation to each row in the second relation. The
    cardinality of the cross product is |rel_a| * |rel_b|.
    
    Parameters:
        rel_a : the first relation.
        rel_b : the second relation.
    
    Returns:
        The cross product between relations rel_a and rel_b.
    """
    return [row_a + row_b for row_a in rel_a for row_b in rel_b]

To test our cross product function take the cross product of actors and movies

In [ ]:
crossproduct(actors, movies)

Finally, lets print the name of actors and the name of movies they have had a role in it.

In [ ]:
# create relation as the crossproduct of actors and movies.
cross_prod = crossproduct(actors, movies)

# select the (actor_id,movie_id) pair from the cross product.
act_mov_pairs = project(cross_prod, [0, 4])

# likewise, get the (actor_name,movie_name) pair from the cross product.
acted_in = project(cross_prod, [1, 2, 5])

# get the roles, but we only need id pairs.
role_list = project(roles, [0, 1])

# generate the relation we need.
rel = [acted_in[i] for i,j in enumerate(act_mov_pairs) if j in role_list]

In [ ]:
rel